In [ ]:
% matplotlib inline

import numpy as np
import numpy.ma as ma
import pandas as pd
from math import *
import matplotlib.pyplot as plt
import scipy.io as sio
from mpl_toolkits.basemap import Basemap, cm
import seawater as sw

import MITgcmutils as mit
from MITgcmutils import cs

from matplotlib import rcParams, rc
rcParams.update({'figure.autolayout': True})
rcParams.update({'font.family': 'serif'})
rcParams.update({'font.serif': 'Times New Roman'})


In [ ]:
# import data needed
# phyto, PP, zoo, MLD, NO3
# CR data
CR_P = np.fromfile('/Users/sclayton/Google Drive/work/comparison/data/monthly/CR/IntPhyto.monthly.1999.data', dtype = '>f')
CR_P = CR_P.reshape(12, 160, 360)
CR_PP = np.fromfile('/Users/sclayton/Google Drive/work/comparison/data/monthly/CR/IntPP.monthly.1999.data', dtype = '>f')
CR_PP = CR_PP.reshape(12, 160, 360)
CR_Z = np.fromfile('/Users/sclayton/Google Drive/work/comparison/data/monthly/CR/IntZoo.monthly.1999.data', dtype = '>f')
CR_Z = CR_Z.reshape(12, 160, 360)
CR_N = np.fromfile('/Users/sclayton/Google Drive/work/comparison/data/monthly/CR/nit.monthly.1999.data', dtype = '>f')
CR_N = CR_N.reshape(12, 160, 360)
CR_mld = np.fromfile('/Users/sclayton/Google Drive/work/comparison/data/monthly/CR/mld.monthly.1999.data', dtype = '>f')
CR_mld = CR_mld.reshape(12, 160, 360)

# HR data
HR_Pfile = '/Users/sclayton/Google Drive/work/comparison/data/monthly/HR/intBio.monthly.1999%02d.data' % m
HR_PPfile = '/Users/sclayton/Google Drive/work/comparison/data/monthly/HR/intPP.monthly.1999%02d.data' % m
HR_mldfile = '/Users/sclayton/Google Drive/work/comparison/data/monthly/HR/MLD.1999%02d.data' % m
HR_Nfile = '/Users/sclayton/Google Drive/work/comparison/data/monthly/HR/SurfNutsZoo.monthly.1999%02d.data' % m
HR_Zfile = '/Users/sclayton/Google Drive/work/comparison/data/monthly/HR/intZoo.monthly.1999%02d.data' % m